### 35.1 Recap of metaclasses

#### Original
Create a metaclass called AutoInit that automatically injects an __init__ method into 
any class that:

Defines a class-level list called init_fields, e.g., ["name", "salary"].

If the __init__ method already exists, the metaclass should not override it.

The injected __init__ should assign values to those fields as instance attributes.

Raise an error if the init_fields list is missing or not a list.

#### Thought through specificaitons

Create a metaclass called AutoInit that automatically injects an __init__ method into 
any class that:

    Spec 1:
    Defines a class-level list called init_fields, e.g., ["name", "salary"].
    Raise an error if the init_fields list is missing or not a list.

    Spec 2:
    If the __init__ method already exists, the metaclass should not override it.
    The injected __init__ should assign values to those fields as instance attributes.
    


In [5]:
# Basic structure of a metaclass
class AutoInit(type):

    def __new__(cls, name, bases, dct):

        return super().__new__(cls, name, bases, dct)

In [ ]:
# Spec 1
class AutoInit(type):

    def __new__(cls, name, bases, dct):

        # check for init_fields
        init_fields = dct.get('init_fields')
        if init_fields is None or not isinstance(init_fields, list):
            raise TypeError(f"Class '{name}' must define class-level list called init_fields")


        return super().__new__(cls, name, bases, dct)

In [10]:
# Spec 1 + Spec 2 (outline)
class AutoInit(type):

    def __new__(cls, name, bases, dct):

        # check for init_fields
        init_fields = dct.get('init_fields')
        if init_fields is None or not isinstance(init_fields, list):
            raise TypeError(f"Class '{name}' must define class-level list called init_fields")

        # check if __init__ is defined, otherwise inject it
        if '__init__' not in dct:
            def __init__(self, *args):
                pass
            dct['__init__'] = __init__


        return super().__new__(cls, name, bases, dct)


In [42]:
# Spec 1 + Spec 2 
class AutoInit(type):

    def __new__(cls, name, bases, dct):

        # check for init_fields
        init_fields = dct.get('init_fields')
        if init_fields is None or not isinstance(init_fields, list):
            raise TypeError(f"Class '{name}' must define class-level list called init_fields")

        # check if __init__ is defined, otherwise inject it
        if '__init__' not in dct:
            def __init__(self, *args):
                if len(args) != len(init_fields):
                    raise ValueError(f"{len(init_fields)} expected, got {len(args)}")
                for field, value in zip(init_fields, args):
                    setattr(self, field, value)
            dct['__init__'] = __init__

        # Just to demonstrate that you can inject any function as you need
        def somefunction(self):
            print("This is somefunction")
        dct['somefunction'] = somefunction


        return super().__new__(cls, name, bases, dct)

In [44]:
class E:
    pass

In [46]:
class E(metaclass=AutoInit):
    pass

TypeError: Class 'E' must define class-level list called init_fields

In [48]:
class Employee(metaclass=AutoInit):
    init_fields = ['name', 'salary']

    def display(self):
        return f"Employee: {self.name}, Salary: {self.salary}"

In [50]:
e = Employee("Anil", 1000000)
e.display()

'Employee: Anil, Salary: 1000000'

In [52]:
dir(Employee)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'display',
 'init_fields',
 'somefunction']

In [54]:
e.somefunction()

This is somefunction


### 35.2 The @cache Decorator

Compute Fibonacci numbers recursively
Compare execution with and without caching

In [61]:
import time
import tracemalloc
from functools import cache  # also use lru_cache -> adv. you can specify max_size

In [63]:
# without cache
def fibogen(n):
    if n < 2:
        return n
    return fibogen(n-1) + fibogen(n-2)

In [65]:
# with cache
@cache
def fibogen_cache(n):
    if n < 2:
        return n
    return fibogen(n-1) + fibogen(n-2)

In [73]:
def measure_execution(func, n):

    tracemalloc.start()
    start_time = time.time()
    result = func(n)
    end_time = time.time()
    current, peak = tracemalloc.get_traced_memory()
    tracemalloc.stop()

    print(f"Funcion: {func.__name__}")
    print(f"Result: {result}")
    print(f"Time taken: {end_time - start_time:.4f} seconds")
    print(f"Memory Used: {current} Bytes; Peak: {peak} Bytes\n")

In [87]:
N = 40
measure_execution(fibogen, N)

Funcion: fibogen
Result: 102334155
Time taken: 19.5016 seconds
Memory Used: 1235 Bytes; Peak: 1683 Bytes



In [89]:
N = 40
measure_execution(fibogen_cache, N)

Funcion: fibogen_cache
Result: 102334155
Time taken: 19.9001 seconds
Memory Used: 149153 Bytes; Peak: 149793 Bytes



In [91]:
N = 38
measure_execution(fibogen_cache, N)

Funcion: fibogen_cache
Result: 39088169
Time taken: 7.6744 seconds
Memory Used: 1857 Bytes; Peak: 2273 Bytes



In [93]:
N = 42
measure_execution(fibogen_cache, N)

Funcion: fibogen_cache
Result: 267914296
Time taken: 54.3010 seconds
Memory Used: 1747 Bytes; Peak: 2419 Bytes



In [95]:
N = 40
measure_execution(fibogen_cache, N)

Funcion: fibogen_cache
Result: 102334155
Time taken: 0.0000 seconds
Memory Used: 0 Bytes; Peak: 0 Bytes



In [97]:
N = 38
measure_execution(fibogen_cache, N)

Funcion: fibogen_cache
Result: 39088169
Time taken: 0.0000 seconds
Memory Used: 0 Bytes; Peak: 0 Bytes



In [99]:
N = 42
measure_execution(fibogen_cache, N)

Funcion: fibogen_cache
Result: 267914296
Time taken: 0.0000 seconds
Memory Used: 0 Bytes; Peak: 0 Bytes



##### See the difference

In [101]:
N = 40
measure_execution(fibogen, N)

Funcion: fibogen
Result: 102334155
Time taken: 20.2619 seconds
Memory Used: 1579 Bytes; Peak: 2283 Bytes



In [103]:
N = 40
measure_execution(fibogen_cache, N)

Funcion: fibogen_cache
Result: 102334155
Time taken: 0.0000 seconds
Memory Used: 0 Bytes; Peak: 0 Bytes



### 35.3 The @total_ordering Decorator

##### Given a class defining one or more rich comparison ordering methods, this class decorator supplies the rest.

Why Use @total_ordering?
Without it, you'd have to implement all 6 comparison methods:

__eq__, __ne__, __lt__, __le__, __gt__, __ge__  -> override all these

With @total_ordering, you only need __eq__ + one of the others, and the rest are auto-generated.

Reference: https://docs.python.org/3/library/functools.html

s1 > s2 # compare the average marks

In [155]:
from functools import total_ordering

@total_ordering
class Student(object):

    def __init__(self, name, marks):
        self.name = name
        self.marks = marks

    def __eq__(self, other):
        if not isinstance(other, Student):
            return NotImplemented
        return self.marks == other.marks

    def __lt__(self, other):
        if not isinstance(other, Student):
            return NotImplemented 
        return self.marks < other.marks

    def __repr__(self):
        return f"{self.name}({self.marks})"

In [157]:
s1 = ("Anil", 398)
s2 = ("Sunil", 407)

In [159]:
s1, s2

(('Anil', 398), ('Sunil', 407))

In [161]:
s1 == s2 # works, because it is implemented

False

In [163]:
s1 < s2 # works, because it is implemented

True

In [165]:
s1 >= s2 # technically this will not work, as it is not implemented

False

In [168]:
s1 <= s2

True

In [170]:
s1 > s2

False

In [172]:
s1 < s2

True

## 35.4 The reduce Function

In [175]:
L = [1, 2, 3, 4, 5] # Reduce to -> 15, window = 2

In [177]:
from functools import reduce
reduce(lambda a, b: a + b, L)

15

In [181]:
from functools import reduce
reduce(lambda a, b: a * b, L)

120